In [5]:
import pandas as pd

# Cargar los datos
df = pd.read_csv("bike_buyers.csv")

# Ver las primeras filas
print("Primeras filas:")
print(df.head())

# Ver la información básica
print("\nInformación del archivo:")
print(df.info())

# Ver si hay datos faltantes
print("\nDatos faltantes por columna:")
print(df.isnull().sum())

# Ver valores más comunes en la columna objetivo
print("\nFrecuencia de Purchased Bike:")
print(df["Purchased Bike"].value_counts())

# Ver frecuencia de columnas categóricas básicas
print("\nFrecuencia de Marital Status:")
print(df["Marital Status"].value_counts())

print("\nFrecuencia de Gender:")
print(df["Gender"].value_counts())


Primeras filas:
      ID Marital Status  Gender   Income  Children        Education  \
0  12496        Married  Female  40000.0       1.0        Bachelors   
1  24107        Married    Male  30000.0       3.0  Partial College   
2  14177        Married    Male  80000.0       5.0  Partial College   
3  24381         Single     NaN  70000.0       0.0        Bachelors   
4  25597         Single    Male  30000.0       0.0        Bachelors   

       Occupation Home Owner  Cars Commute Distance   Region   Age  \
0  Skilled Manual        Yes   0.0        0-1 Miles   Europe  42.0   
1        Clerical        Yes   1.0        0-1 Miles   Europe  43.0   
2    Professional         No   2.0        2-5 Miles   Europe  60.0   
3    Professional        Yes   1.0       5-10 Miles  Pacific  41.0   
4        Clerical         No   0.0        0-1 Miles   Europe  36.0   

  Purchased Bike  
0             No  
1             No  
2             No  
3            Yes  
4            Yes  

Información del archi

In [6]:
# Eliminación de filas con valores nulos
df_cleaned = df.dropna().copy()

print(f"Filas originales: {len(df)}")
print(f"Filas después de la limpieza: {len(df_cleaned)}")
print(f"Registros eliminados: {len(df) - len(df_cleaned)}")
# Verificar que no quedan nulos
print("\nVerificación de valores nulos:")
print(df_cleaned.isnull().sum().sum())

Filas originales: 1000
Filas después de la limpieza: 952
Registros eliminados: 48

Verificación de valores nulos:
0


You can upload the `bike_buyers.csv` file directly to your Colab environment. Here's how:

1.  **Click on the folder icon** on the left sidebar (File Browser).
2.  **Click on the 'Upload to session storage' icon** (the icon with an arrow pointing upwards).
3.  **Select `bike_buyers.csv`** from your local machine and click 'Open'.

Alternatively, you can use the following Python code to upload the file:

In [8]:
import pandas as pd
import numpy as np


# Estas son las columnas que queremos revisar
numeric_cols = ['Income', 'Age']
outlier_limits = {}
df_cleaned = df.dropna().copy()

for col in numeric_cols:

    # A. Calculamos el "Rango Normal" (donde está el 50% de los datos)
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1

    # B. Definimos los "Topes Extremos" (usando 3 veces el IQR)
    # Todo lo que esté fuera de aquí, lo consideramos outlier.
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    outlier_limits[col] = (lower_bound, upper_bound)

    print(f"\n--- Límites de Detección para {col} ---")
    print(f"Límite Inferior (Todo debajo de {lower_bound:.2f} es raro):")
    print(f"Límite Superior (Todo por encima de {upper_bound:.2f} es raro):")

    # Contamos cuántos valores son raros (outliers)
    outliers_count = len(df_cleaned[(df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)])
    print(f"Clientes 'raros' detectados: {outliers_count}")


# Hacemos una copia para aplicar los cambios y crear el DataFrame final limpio
df_treated = df_cleaned.copy()

print("\n--- Poniendo topes (Capping) a los números extremos ---")

for col in numeric_cols:
    lower_limit, upper_limit = outlier_limits[col]

    # LÓGICA CAPPING (Reemplazar los valores por el tope)

    # Si el valor actual es MAYOR al límite, lo reemplazamos con el LÍMITE SUPERIOR
    df_treated[col] = np.where(
        df_treated[col] > upper_limit,
        upper_limit,
        df_treated[col]  # Si no es mayor, se deja el valor original
    )

    # Si el valor actual es MENOR al límite, lo reemplazamos con el LÍMITE INFERIOR
    df_treated[col] = np.where(
        df_treated[col] < lower_limit,
        lower_limit,
        df_treated[col] # Si no es menor, se deja el valor original
    )

    print(f"Topes aplicados a {col}.")

print("\n--- Verificando si los valores extremos desaparecieron ---")
# Mostramos las estadísticas para ver si los valores Máximos/Mínimos cambiaron
print(df_treated[numeric_cols].describe())


--- Límites de Detección para Income ---
Límite Inferior (Todo debajo de -90000.00 es raro):
Límite Superior (Todo por encima de 190000.00 es raro):
Clientes 'raros' detectados: 0

--- Límites de Detección para Age ---
Límite Inferior (Todo debajo de -16.00 es raro):
Límite Superior (Todo por encima de 103.00 es raro):
Clientes 'raros' detectados: 0

--- Poniendo topes (Capping) a los números extremos ---
Topes aplicados a Income.
Topes aplicados a Age.

--- Verificando si los valores extremos desaparecieron ---
              Income         Age
count     952.000000  952.000000
mean    55903.361345   44.256303
std     30845.483596   11.428167
min     10000.000000   25.000000
25%     30000.000000   35.000000
50%     60000.000000   43.000000
75%     70000.000000   52.000000
max    170000.000000   89.000000


In [9]:
# REQUERIMIENTO 3: Creación de la variable 'Con_hijos'
# Si: Children > 0, No: Children = 0
df_cleaned['Con_hijos'] = np.where(df_cleaned['Children'] > 0, 'Si', 'No')

# REQUERIMIENTO 4: Creación de la variable 'Con_vehiculo'
# Si: Cars > 0, No: Cars = 0
df_cleaned['Con_vehiculo'] = np.where(df_cleaned['Cars'] > 0, 'Si', 'No')

print("\n--- Muestra con nuevas variables ---")
print(df_cleaned[['Children', 'Con_hijos', 'Cars', 'Con_vehiculo']].head())


--- Muestra con nuevas variables ---
   Children Con_hijos  Cars Con_vehiculo
0       1.0        Si   0.0           No
1       3.0        Si   1.0           Si
2       5.0        Si   2.0           Si
4       0.0        No   0.0           No
5       2.0        Si   0.0           No


In [10]:
income_by_purchase = df_cleaned.groupby('Purchased Bike')['Income'].mean().reset_index()
income_by_purchase['Income'] = income_by_purchase['Income'].round(2)

print("\n1. Promedio de Ingresos según Compra de Bicicleta:")
print(income_by_purchase)


1. Promedio de Ingresos según Compra de Bicicleta:
  Purchased Bike    Income
0             No  54455.65
1            Yes  57478.07


In [11]:
income_by_marital_status = df_cleaned.groupby('Marital Status')['Income'].mean().reset_index()
income_by_marital_status['Income'] = income_by_marital_status['Income'].round(2)
income_by_marital_status = income_by_marital_status.rename(columns={'Marital Status': 'Estado Civil'})

print("\n2. Promedio de Ingresos según Estado Civil:")
print(income_by_marital_status)


2. Promedio de Ingresos según Estado Civil:
  Estado Civil    Income
0      Married  58359.07
1       Single  52972.35


In [12]:
children_by_education = df_cleaned[df_cleaned['Con_hijos'] == 'Si'].groupby('Education')['Children'].mean().reset_index()
children_by_education['Children'] = children_by_education['Children'].round(2)
children_by_education = children_by_education.rename(columns={'Children': 'Promedio de Hijos', 'Education': 'Nivel Educativo'})

print("\n3. Clientes con Hijos: Promedio de Hijos según Nivel Educativo:")
print(children_by_education)


3. Clientes con Hijos: Promedio de Hijos según Nivel Educativo:
       Nivel Educativo  Promedio de Hijos
0            Bachelors               2.53
1      Graduate Degree               2.69
2          High School               2.77
3      Partial College               2.58
4  Partial High School               2.78


In [13]:
cars_by_occupation = df_cleaned[df_cleaned['Con_vehiculo'] == 'Si'].groupby('Occupation')['Cars'].mean().reset_index()
cars_by_occupation['Cars'] = cars_by_occupation['Cars'].round(2)
cars_by_occupation = cars_by_occupation.rename(columns={'Cars': 'Promedio de Vehículos'})

print("\n4. Clientes con Vehículo: Promedio de Vehículos según Ocupación:")
print(cars_by_occupation)


4. Clientes con Vehículo: Promedio de Vehículos según Ocupación:
       Occupation  Promedio de Vehículos
0        Clerical                   1.49
1      Management                   2.34
2          Manual                   1.45
3    Professional                   2.12
4  Skilled Manual                   1.72


In [14]:
age_by_home_owner = df_cleaned.groupby('Home Owner')['Age'].mean().reset_index()
age_by_home_owner['Age'] = age_by_home_owner['Age'].round(2)
age_by_home_owner = age_by_home_owner.rename(columns={'Home Owner': 'Propietario Vivienda', 'Age': 'Promedio de Edad'})

print("\n5. Promedio de Edad según Propietario de Vivienda:")
print(age_by_home_owner)


5. Promedio de Edad según Propietario de Vivienda:
  Propietario Vivienda  Promedio de Edad
0                   No             42.39
1                  Yes             45.11


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. CARGA Y LIMPIEZA DE DATOS (Preparación del set para modelado)
# ------------------------------------------------------------------
# Cargar el archivo original
df = pd.read_csv("bike_buyers.csv")

# Eliminar filas con valores nulos (la técnica de omisión utilizada antes)
df_model = df.dropna().copy()
print(f"Filas listas para modelado: {len(df_model)}")

# Convertir la variable objetivo (TARGET) de texto a número (0 y 1)
# El modelo necesita números: 'Yes' (compró) = 1, 'No' (no compró) = 0
df_model['Purchased Bike'] = df_model['Purchased Bike'].map({'Yes': 1, 'No': 0})

Filas listas para modelado: 952


In [16]:
# 2. SEPARACIÓN DE VARIABLES Y CODIFICACIÓN
# ------------------------------------------------------------------

# Definir la variable a predecir (Target) y las variables predictoras (Features)
X = df_model.drop('Purchased Bike', axis=1) # Todas las columnas EXCEPTO el Target
y = df_model['Purchased Bike']              # Solo el Target

# Identificar todas las columnas de texto (categóricas) para codificar
categorical_features = X.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding (Convierte texto en columnas binarias)
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Eliminamos la columna ID que no es útil para la predicción
X_encoded = X_encoded.drop('ID', axis=1)

print(f"\nNúmero de columnas después de la codificación: {X_encoded.shape[1]}")


Número de columnas después de la codificación: 21


In [17]:
# 3. PLAN DE PRUEBA: DIVISIÓN ENTRENAMIENTO/PRUEBA
# ------------------------------------------------------------------

# Separación 70% para entrenar, 30% para probar
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.3,       # 30% de los datos para prueba
    random_state=42,     # Fija la semilla para que los resultados sean repetibles
    stratify=y           # Asegura que la proporción de 'Yes'/'No' sea similar en ambos conjuntos
)

print(f"\nDatos de Entrenamiento (70%): {len(X_train)} filas")
print(f"Datos de Prueba (30%): {len(X_test)} filas")


# 4. CONSTRUIR EL MODELO: REGRESIÓN LOGÍSTICA
# ------------------------------------------------------------------

# Crear una instancia del modelo de Regresión Logística (el clasificador)
log_model = LogisticRegression(max_iter=1000)

# ENTRENAR EL MODELO: Aquí es donde el modelo 'aprende' de los datos
log_model.fit(X_train, y_train)

print("\n=> Modelo de Regresión Logística entrenado con éxito.")


Datos de Entrenamiento (70%): 666 filas
Datos de Prueba (30%): 286 filas

=> Modelo de Regresión Logística entrenado con éxito.


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
# 5. EVALUACIÓN DEL MODELO
# ------------------------------------------------------------------

# El modelo hace predicciones sobre los datos de prueba (los datos que NUNCA ha visto)
y_pred = log_model.predict(X_test)

# Medimos la calidad del modelo: ¿Qué tan a menudo acertó? (Accuracy Score)
accuracy = accuracy_score(y_test, y_pred)

print("-" * 50)
print(f"PRECISIÓN DEL MODELO (Accuracy): {accuracy:.4f}")
print("-" * 50)

# Muestra de las primeras 10 predicciones vs. la realidad
comparison = pd.DataFrame({
    'Real': y_test.head(10),
    'Predicción': y_pred[:10]
})
print("\nComparación de las primeras 10 predicciones:")
print(comparison)

--------------------------------------------------
PRECISIÓN DEL MODELO (Accuracy): 0.6643
--------------------------------------------------

Comparación de las primeras 10 predicciones:
     Real  Predicción
220     1           1
303     0           0
626     0           0
697     0           0
447     1           1
776     1           1
588     1           0
680     0           0
727     1           1
80      1           0


In [19]:
df_treated.to_csv('bike_buyers_LIMPIO.csv', index=False)
from google.colab import files
files.download('bike_buyers_LIMPIO.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving bike_buyers.csv to bike_buyers.csv
User uploaded file "bike_buyers.csv" with length 87182 bytes
